In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers, optimizers
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from keras import backend as K
from sklearn.metrics import log_loss

from subprocess import check_output
print(check_output(["ls", "../Training"]).decode("utf8"))

Using TensorFlow backend.


Untitled.ipynb
WCities.csv
WGameCities.csv
wModel.py
WNCAATourneyCompactResults.csv
WNCAATourneySeeds.csv
WNCAATourneySlots.csv
WRegularSeasonCompactResults.csv
WSampleSubmissionStage1.csv
WSeasons.csv
WTeams.csv
WTeamSpellings.csv



In [2]:
# Load data
data_dir = '../Training/'
df_seeds = pd.read_csv(data_dir + 'WNCAATourneySeeds.csv')
df_tour =  pd.read_csv(data_dir + 'WNCAATourneyCompactResults.csv')

# extract seeds as integers
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_seeds.head()

# Drop extra cols
df_tour.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.shape

(1260, 3)

In [3]:
# Merge seeds and concatenate
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat1 = df_concat[df_concat.Season<2014]
df_concat2 =  df_concat[df_concat.Season>=2014] 

# Win and loss dfs
df_wins = pd.DataFrame()
df_wins['SeedDiff'] = df_concat1['SeedDiff']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['SeedDiff'] = -df_concat1['SeedDiff']
df_losses['Result'] = 0

df_train = pd.concat((df_wins, df_losses))

In [4]:
# Training set
df_winTest= pd.DataFrame()
df_winTest['SeedDiff'] = df_concat2['SeedDiff']
df_winTest['Result'] = 1

df_lossesTest = pd.DataFrame()
df_lossesTest['SeedDiff'] = -df_concat2['SeedDiff']
df_lossesTest['Result'] = 0

df_test = pd.concat((df_winTest, df_lossesTest))


In [5]:
df_train.head()

,SeedDiff,Result
0,-13,1
1,-11,1
2,-13,1
3,-3,1
4,3,1


In [12]:
df_test.tail()

,SeedDiff,Result
1255,9,0
1256,2,0
1257,-1,0
1258,1,0
1259,1,0


In [15]:
# Select training and test data
X_train = df_train.iloc[:,0].values.reshape(-1,1)
y_train = df_train.Result.values
xDim = np.shape(X_train)[1]
X_train, y_train = shuffle(X_train, y_train)
X_test = df_test.iloc[:,0].values.reshape(-1,1)
y_test = df_test.Result.values

In [16]:
print(X_train)

[[-13]
 [ -1]
 [ -1]
 ...
 [ 13]
 [  3]
 [  1]]


In [9]:
# Train the model
dropRate = 0.3
numBatch = 20
numEpoch = 120
learningRate = 1e-4

# MLP model
MLP = Sequential()
MLP.name = 'MLP'
MLP.add(Dense(5, input_dim=xDim, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(200,kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(200, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))

# Compile model
adam = optimizers.Adam(lr=learningRate, amsgrad=True)
MLP.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [11]:
# Fit model
MLP.fit(X_train, y_train, epochs=numEpoch, batch_size=numBatch, verbose=1)

Epoch 1/120
2016/2016 [==============================] - 0s 141us/step - loss: 0.0109 - acc: 0.9980
Epoch 2/120
2016/2016 [==============================] - 0s 125us/step - loss: 0.0061 - acc: 0.9995
Epoch 3/120
2016/2016 [==============================] - 0s 127us/step - loss: 0.0102 - acc: 0.9980
Epoch 4/120
2016/2016 [==============================] - 0s 134us/step - loss: 0.0032 - acc: 1.0000
Epoch 5/120
2016/2016 [==============================] - 0s 131us/step - loss: 0.0062 - acc: 0.9990
Epoch 6/120
2016/2016 [==============================] - 0s 125us/step - loss: 0.0082 - acc: 0.9985
Epoch 7/120
2016/2016 [==============================] - 0s 128us/step - loss: 0.0036 - acc: 0.9995
Epoch 8/120
2016/2016 [==============================] - 0s 129us/step - loss: 0.0019 - acc: 1.0000
Epoch 9/120
2016/2016 [==============================] - 0s 127us/step - loss: 0.0105 - acc: 0.9975
Epoch 10/120
2016/2016 [==============================] - 0s 128us/step - loss: 0.0071 - acc: 0.9985

KeyboardInterrupt: 

In [ ]:
# Test model
MLP.evaluate(X_train, y_train)

In [ ]:
# Make Predictions 
preds = MLP.predict_proba(X_test)[:,1]

clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub.head()